## Training a model by fine tuning the top layers of pre-trained network
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [1]:
import numpy as np
import math
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
from keras.models import Model

C:\Users\vipul\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
# dimensions of our images.
img_width, img_height = 256, 256

train_data_dir = r"C:\Users\**\Desktop\jupyter\DL\CNN egs\Data\train"
validation_data_dir = r"C:\Users\**\Desktop\jupyter\DL\CNN egs\Data\validation"
nb_train_samples = 2000
nb_validation_samples = 600
epochs = 10
batch_size = 32

In [3]:
#Top model of 2nd_convet
top_model_weights_path = '2ndconvnet_bottleneck_features_weights.h5'

In [4]:
# build the VGG16 network
vgg16_model = applications.VGG16(include_top=False, input_shape=(img_width,img_height,3))
print('Model loaded.')

Model loaded.


In [5]:
# build a classifier model to put on top of the convolutional model 
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [6]:
#note that it is necessary to start with a fully-trained classifier, including the top classifier, in order to successfully do fine-tuning
#loading weights on the top model
top_model.load_weights(top_model_weights_path)

In [7]:
# add the top_model on top of the convolutional base
model = Model(inputs=vgg16_model.input, outputs=top_model(vgg16_model.output))

In [8]:
# set the first 15 layers (up to the last conv block) to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [9]:
# compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [10]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

model.summary()

Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
___________________________________________________________

In [16]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)

Epoch 1/10
 - 128s - loss: 0.1664 - acc: 0.9476 - val_loss: 0.2178 - val_acc: 0.9097
Epoch 2/10
 - 120s - loss: 0.1255 - acc: 0.9491 - val_loss: 0.1943 - val_acc: 0.9132
Epoch 3/10
 - 120s - loss: 0.1223 - acc: 0.9627 - val_loss: 0.2142 - val_acc: 0.9201
Epoch 4/10
 - 120s - loss: 0.1176 - acc: 0.9551 - val_loss: 0.1963 - val_acc: 0.9167
Epoch 5/10
 - 120s - loss: 0.1006 - acc: 0.9667 - val_loss: 0.1826 - val_acc: 0.9201
Epoch 6/10
 - 121s - loss: 0.0863 - acc: 0.9703 - val_loss: 0.1712 - val_acc: 0.9201
Epoch 7/10
 - 121s - loss: 0.0828 - acc: 0.9678 - val_loss: 0.1853 - val_acc: 0.9271
Epoch 8/10
 - 120s - loss: 0.0771 - acc: 0.9718 - val_loss: 0.1711 - val_acc: 0.9288
Epoch 9/10
 - 120s - loss: 0.0951 - acc: 0.9667 - val_loss: 0.1679 - val_acc: 0.9306
Epoch 10/10
 - 120s - loss: 0.0651 - acc: 0.9738 - val_loss: 0.1773 - val_acc: 0.9306


In [17]:
model.save_weights('3rd_ConvNet_weights.h5')

* Here the validation accuracy is even more, almost **93%**. which is 4% more than the 2nd_convnet , you can furthur increase the accuracy by providing more data and training more layers etc.